<font color = 'blue'><font size = 5> **Final Project Submission**

Please fill out:
* Student name:                        Matan Gavish 
* Student pace:                        part time
* Scheduled project review date/time: 
* Instructor name:                     Joe San Pietro 
* Blog post URL:

Methodology<br>


## <font color = 'green'> Import Libraries 

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', None) # shows me all columns instead of truncating and dot dot dot

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')

from scipy.stats import kurtosis, skew
import scipy.stats as stats

from math import sqrt

import statsmodels.formula.api as smf

## <font color = 'green'> pull in cleaned data set

In [2]:
df = pd.read_csv("cleaned_kc_housing_data.csv")
df.head()

,Unnamed: 0,date_sold,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0,10/13/2014,221900,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,1,12/9/2014,538000,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,2,2/25/2015,180000,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,3,12/9/2014,604000,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,4,2/18/2015,510000,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [3]:
df.drop(columns = 'Unnamed: 0', inplace = True) #drop Unnamed: 0 column
df.drop(columns = 'sqft_living', inplace = True)
df.drop(columns = 'sqft_above', inplace = True)
df.drop(columns = 'sqft_living15', inplace = True)
df.drop(columns = 'sqft_lot15', inplace = True)
df.drop(columns = 'waterfront', inplace = True)
df.drop(columns = 'condition', inplace = True)
df.drop(columns = 'sqft_lot', inplace = True)
df.drop(columns = 'yr_built', inplace = True)
df.drop(columns = 'yr_renovated', inplace = True)
df.drop(columns = 'long', inplace = True)

###### remove outliers

In [4]:
df = df[df.bedrooms < 10]
df = df[df.bathrooms < 5]
df = df[df.sqft_basement < 2000]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21407 entries, 0 to 21533
Data columns (total 10 columns):
date_sold        21407 non-null object
price            21407 non-null int64
bedrooms         21407 non-null int64
bathrooms        21407 non-null float64
floors           21407 non-null float64
view             21407 non-null int64
grade            21407 non-null int64
sqft_basement    21407 non-null int64
zipcode          21407 non-null int64
lat              21407 non-null float64
dtypes: float64(3), int64(6), object(1)
memory usage: 1.8+ MB


#### bin(?) zip codes into towns and merge with df

In [6]:
df1 = pd.read_csv("zipcode_town.csv")

In [7]:
df1.head()

,zipcode,town
0,98001,Auburn
1,98002,Auburn
2,98003,Federal Way
3,98004,Bellevue
4,98005,Bellevue


### merge both dataframes on zip code and pull in town

In [8]:
df2 = df.merge(df1, on = "zipcode", how = "left")

In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21407 entries, 0 to 21406
Data columns (total 11 columns):
date_sold        21407 non-null object
price            21407 non-null int64
bedrooms         21407 non-null int64
bathrooms        21407 non-null float64
floors           21407 non-null float64
view             21407 non-null int64
grade            21407 non-null int64
sqft_basement    21407 non-null int64
zipcode          21407 non-null int64
lat              21407 non-null float64
town             21407 non-null object
dtypes: float64(3), int64(6), object(2)
memory usage: 2.0+ MB


In [10]:
df2.tail()

,date_sold,price,bedrooms,bathrooms,floors,view,grade,sqft_basement,zipcode,lat,town
21402,5/21/2014,360000,3,2.50,3.0,0,8,0,98103,47.6993,Seattle
21403,2/23/2015,400000,4,2.50,2.0,0,8,0,98146,47.5107,Seattle
21404,6/23/2014,402101,2,0.75,2.0,0,7,0,98144,47.5944,Seattle
21405,1/16/2015,400000,3,2.50,2.0,0,8,0,98027,47.5345,Issaquah
21406,10/15/2014,325000,2,0.75,2.0,0,7,0,98144,47.5941,Seattle


#### remove zipcode, lat and long

In [11]:
df = df2.drop(columns = "zipcode")

In [12]:
df = df.drop(columns = ['lat'])

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21407 entries, 0 to 21406
Data columns (total 9 columns):
date_sold        21407 non-null object
price            21407 non-null int64
bedrooms         21407 non-null int64
bathrooms        21407 non-null float64
floors           21407 non-null float64
view             21407 non-null int64
grade            21407 non-null int64
sqft_basement    21407 non-null int64
town             21407 non-null object
dtypes: float64(2), int64(5), object(2)
memory usage: 1.6+ MB


In [14]:
df.head()

,date_sold,price,bedrooms,bathrooms,floors,view,grade,sqft_basement,town
0,10/13/2014,221900,3,1.00,1.0,0,7,0,Seattle
1,12/9/2014,538000,3,2.25,2.0,0,7,400,Seattle
2,2/25/2015,180000,2,1.00,1.0,0,6,0,Kenmore
3,12/9/2014,604000,4,3.00,1.0,0,7,910,Seattle
4,2/18/2015,510000,3,2.00,1.0,0,8,0,Sammamish


## <font color = 'green'> **Begin Modeling**

In [15]:
from sklearn.preprocessing import LabelEncoder  # import LabelEncoder
lb_make = LabelEncoder()  # creating variable to perform action -- why? brevity or serves a greater purpose?

In [16]:
encoded = pd.DataFrame()  # label encoded DataFrame

encoded["bedrooms"] = lb_make.fit_transform(df.bedrooms)
encoded["bathrooms"] = lb_make.fit_transform(df.bathrooms)
encoded["floors"] = lb_make.fit_transform(df.floors)
encoded["view"] = lb_make.fit_transform(df.view)
encoded["grade"] = lb_make.fit_transform(df.grade)


encoded["price"] = df.price

In [17]:
encoded.bedrooms.unique()

array([2, 1, 3, 4, 0, 5, 6, 7, 8], dtype=int64)

 ##### use encoded df in ols model 

In [18]:
col_names = encoded.describe().drop([ 'price'],axis =1)
results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p-value' ]]
for idx, val in enumerate(col_names):
    print ("Kings County: price~" + val)
    print ("------------------------------")

    f = 'price~' + val
    model = smf.ols(formula=f, data=df).fit()
    X_new = pd.DataFrame({val: [encoded[val].min(), encoded[val].max()]});
    preds = model.predict(X_new)
    results.append([val, model.rsquared, model.params[0], model.params[1], model.pvalues[1] ])
    print(results[idx+1])

Kings County: price~bedrooms
------------------------------
['bedrooms', 0.09745046050698891, 135275.4799110159, 118200.29232916248, 0.0]
Kings County: price~bathrooms
------------------------------
['bathrooms', 0.2559001260852414, 50322.60647925375, 229411.46972214052, 0.0]
Kings County: price~floors
------------------------------
['floors', 0.07329473872733527, 280210.13783926715, 168970.25028226344, 0.0]
Kings County: price~view
------------------------------
['view', 0.15162064551555132, 493015.9248895022, 174377.66705588278, 0.0]
Kings County: price~grade
------------------------------
['grade', 0.45064336128390015, -966873.0933119348, 196201.71356855152, 0.0]


In [19]:
pd.DataFrame(results)

,0,1,2,3,4
0,ind_var,r_squared,intercept,slope,p-value
1,bedrooms,0.0974505,135275,118200,0
2,bathrooms,0.2559,50322.6,229411,0
3,floors,0.0732947,280210,168970,0
4,view,0.151621,493016,174378,0
5,grade,0.450643,-966873,196202,0


In [20]:
from sklearn.model_selection import train_test_split


y = encoded[["price"]]
X = encoded.drop(["price"], axis=1)

In [21]:
X = df[['bedrooms', 'bathrooms', 'floors', 'view', 'grade']]
y = df["price"]

#### train-test-split
split the sample data into sub-sets to fit and then test the model

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [23]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

17125 4282 17125 4282


In [24]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

In [25]:
linreg.score(X,y)

0.5196152164027481

In [26]:
train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test

In [27]:
mse_train = np.sum((y_train-y_hat_train)**2)/len(y_train)
mse_test =np.sum((y_test-y_hat_test)**2)/len(y_test)
print('Train Mean Squarred Error:', mse_train)
print('Test Mean Squarred Error:', mse_test)

Train Mean Squarred Error: 54962206549.39629
Test Mean Squarred Error: 52811857472.76364


(redundant?)
sklearn mean squared error (different way of measuring MSE)

In [28]:
from sklearn.metrics import mean_squared_error

train_mse = mean_squared_error(y_train, y_hat_train)
test_mse = mean_squared_error(y_test, y_hat_test)
print('Train Mean Squarred Error:', train_mse)
print('Test Mean Squarred Error:', test_mse)

Train Mean Squarred Error: 54962206549.39629
Test Mean Squarred Error: 52811857472.76364
